# PySpark Cookbook

### Tomasz Drabas, Denny Lee
#### Version: 0.1
#### Date: 3/10/2018

# Introducing Transformers

# Introducing Estimators

# Introducing Pipelines

# Predicting forest coverage type

# Estimating forest elevation

# Clustering forest cover type

# Selecting the most predictable features

# Tuning hyper parameters

# Calculating performance characteristics

# Extracting features

# Discretizing continuous variables

# Standardizing continuous variables

# Topic mining